In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')

In [ ]:
!pip install -U keras

In [ ]:
!pip install -U -q PyDrive


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
download = drive.CreateFile({'id' : "19Tm5sZ-HSeCoRanmidNWo85LjL715rvE"})
download.GetContentFile("finalset.zip")

In [ ]:
!unzip finalset.zip

In [ ]:
!ls

In [ ]:
import tensorflow as tf
import cv2 ##??
import numpy as np
import pandas as pd

from tensorflow.python.keras import layers
from tensorflow.python.keras import models
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn.utils import class_weight


from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.applications.inception_resnet_v2 import preprocess_input
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.callbacks import CSVLogger

from tensorflow.keras.models import load_model




In [ ]:
#funkce presouvajici vsechny kategorie z jedne slozky do jine (napr z val do train) v pripade, ze je to potreba
def movefromto(srcdir, destdir):
  import math
  import shutil
  import os
  from os import listdir
  from os.path import isfile, join
  import random
  categories = os.listdir(srcdir)
  for category in categories:
    srcpath = join(srcdir, category, "")
    files = [f for f in listdir(srcpath) if isfile(join(srcpath, f))]
    random.seed(55)
    #totalLen = len(files)
    #shuffledval = random.sample(files, math.floor(totalLen*perct))

    for file in files:
      shutil.move(srcpath + file, join(destdir, category, ""))

In [ ]:
import math
import shutil
import os
from os import listdir
from os.path import isfile, join
import random
def split(valperct, testperct, traindir, valdir, testdir):
  categories = os.listdir(traindir)
  for category in categories:
    srcpath = join(traindir, category, "")
    files = [f for f in listdir(srcpath) if isfile(join(srcpath, f))]
    random.seed(55)
    totalLen = len(files)
    shuffledval = random.sample(files, math.floor(totalLen*valperct))
    #remove items used for val from the list, keep total len
    #odstranit data pouzita pro val, ulozit si celkovou delku
    new_list = [item for item in files if item not in shuffledval]
    shuffledtrain =  random.sample(new_list, math.floor(totalLen*testperct))

    for file in shuffledval:
      shutil.move(srcpath + file, join(valdir, category, ""))
    for file in shuffledtrain:
      shutil.move(srcpath + file, join(testdir, category, ""))



In [ ]:
#traindir = "./WD/WD/train/"
#valdir = "./WD/WD/val/"
#testdir = "./WD/WD/test/"
#valsplit = 0.1
#testsplit = 0.1
#split(valsplit, testsplit, traindir, valdir, testdir)

In [ ]:
os.getcwd()

In [ ]:
pretrained = InceptionResNetV2(weights = "imagenet", include_top =False, input_shape = (256,256,3))
#from tensorflow.python.keras.applications.mobilenet_v2 import MobileNetV2
#pretrained = MobileNetV2(weights = "imagenet", include_top =False, input_shape = (256,256,3))
#pretrained.trainable = True


In [ ]:
pretrained.trainable = False

In [ ]:

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30, min_delta = 0.0001)

In [ ]:
filepath = "/content/drive/My Drive/Python/Run-improvement-{epoch:02d}-{val_accuracy:.4f}.hdf5"
checkp = ModelCheckpoint(filepath, monitor = "val_accuracy", mode = "max", save_best_only=True)
logging = CSVLogger('/content/drive/My Drive/Python/history.csv', append=False, separator=";")

In [ ]:
model = tf.python.keras.Sequential()
model.add(pretrained)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation = "relu"))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='sigmoid'))

In [ ]:
model.compile(optimizer =tf.keras.optimizers.Adam(learning_rate = 0.0001), loss = "binary_crossentropy", metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
datagen_augment = ImageDataGenerator(preprocessing_function = preprocess_input, horizontal_flip = True, 
                             brightness_range=[0.8, 1.2],
                             zoom_range = [0.6, 1.0])

In [ ]:
train_generator = datagen_augment.flow_from_directory(
        directory = './finalset2/train',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')

In [ ]:
#pro val a test neni potreba pouzivat augmentace
datagen = ImageDataGenerator(preprocessing_function = preprocess_input)

In [ ]:

val_generator = datagen.flow_from_directory(
        directory = './finalset2/val',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical')
test_generator = datagen.flow_from_directory(
        directory = './finalset2/test',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)

In [ ]:
#nastavit class indexy pro class vahy do dict
class_weights_dict = {}
class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)
class_array = np.unique(train_generator.classes)
for  K, V in zip(class_array, class_weights):
  class_weights_dict[K] = V
print(class_weights_dict)
print(class_weights)

In [ ]:
#vytvoreni souboru dict.json
class_map = val_generator.class_indices
import json
json = json.dumps(class_map)
f = open("dict.json","w")
f.write(json)
f.close()

In [ ]:
class_map

In [ ]:
#kontrola pripojení ke gpu
tf.test.gpu_device_name()

In [ ]:
ntrain = train_generator.n
nval = val_generator.n
ntest = test_generator.n
batch_size = 32 


In [ ]:
fit = model.fit(train_generator,
                epochs = 100,
                validation_data = val_generator,
                callbacks = [es, checkp, logging],
                class_weight = class_weights_dict,
                shuffle = True,
                batch_size = batch_size 
                )

In [ ]:
np.save('/content/drive/My Drive/Python/finalmodelhistory.npy',fit.history)

In [ ]:
model.save('/content/drive/My Drive/Python/TestRun-final-model.h5')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fit.history["accuracy"])
plt.plot(fit.history["val_accuracy"])


plt.legend(["train", "val"])
plt.show()

In [ ]:
plt.plot(fit.history["loss"])
plt.plot(fit.history["val_loss"])


plt.legend(["train", "val"])
plt.show()

In [ ]:
plt.plot(fit.history["loss"])
plt.plot(fit.history["val_loss"])
plt.plot(fit.history["accuracy"])
plt.plot(fit.history["val_accuracy"])

plt.legend(["loss", "val_loss"])
plt.show()

In [ ]:
from tensorflow.python.keras.preprocessing import image
img = image.load_img('./WD/WD/test/bedroom/00000003.jpg', target_size=(256, 256))

imaget = image.img_to_array(img)
imaget = np.expand_dims(imaget, axis=0) 
img = datagen.standardize(imaget) 
preds = model.predict(imaget)
preds

In [ ]:
preds = model.predict(test_generator, batch_size=32, steps = ntest // batch_size, verbose = 1)

In [ ]:
datagen = ImageDataGenerator(preprocessing_function = preprocess_input)
test_generator = datagen.flow_from_directory(
        directory = './finalset2/test',
        target_size=(256, 256),
        batch_size=32,
        class_mode='categorical',
        shuffle = False)

In [ ]:
#vyhodnoceni jednoho modelu
model.evaluate(test_generator, batch_size=32, steps = ntest // batch_size, verbose = 1 )

In [ ]:
import csv
from tensorflow.keras.models import load_model
import os
import random
random.seed(55)
#testovaci vsech modelu podle pojmenovani, vystup do csv souboru
modelsTesting = []
for file in os.listdir('/content/drive/My Drive/Python/'):
  if file.startswith("Run3"):
    modelsTesting.append(file)
modelsTesting

#
for modelName in modelsTesting:
  model_for_testing = load_model("/content/drive/My Drive/Python/" + modelName)
  print("Currently testing: " + modelName)
  test = model_for_testing.evaluate(test_generator, batch_size=32, verbose = 1 )
  test.insert(0, modelName)

  with open("/content/drive/My Drive/Python/testingResults.csv","a", newline="") as fd:
      wr = csv.writer(fd, dialect="excel")
      wr.writerow(test)


In [ ]:
#predikce pro konkretni model
from tensorflow.keras.models import load_model
model = load_model("/content/drive/My Drive/Python/model.hdf5")

In [ ]:
predictions = model.predict(test_generator, batch_size= 32, verbose = 1)

In [ ]:
len(predictions)

In [ ]:
predictedClasses = np.argmax(predictions, axis=1)

In [ ]:
len(predictedClasses)

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
cm = confusion_matrix(test_generator.classes, predictedClasses, normalize = "true")
plt.imshow(cm, cmap=plt.cm.Blues)

In [ ]:
#plt.imshow(cm, cmap=plt.cm.Blues)
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
def plot_conf_matrix(Y_true, Y_pred, normalize):
  labels = list(test_generator.class_indices.keys())
  if(normalize == True):
    cm = confusion_matrix(test_generator.classes, predictedClasses, normalize = "true")
  else: 
    cm = confusion_matrix(test_generator.classes, predictedClasses)
  fig = plt.figure()
  ax = fig.add_subplot(111)
  cax = ax.matshow(cm, cmap=plt.cm.Blues)
  plt.xlabel("Předikovaná třída")
  plt.ylabel("Skutečná třída")
  #plt.xticks([], [])
  #plt.yticks([], [])
  ax.set_xticklabels([''] + labels, rotation = 45)
  ax.set_yticklabels([''] + labels) 
  #plt.xticks(np.arange(len(labels)), labels)
  ax.tick_params(axis="x", bottom=True, labelbottom= True, top = False, labeltop = False)
  threshhold = cm.max() / 2
  if (normalize == True):
    numberFormat = ".2f"
    plt.title('Normalizovaná matice záměn')    
  else: 
    numberFormat = "d"
    plt.title('Matice záměn')    
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      ax.text(j, i, format(cm[i, j], numberFormat), ha="center", va="center", color="white" if cm[i, j] > threshhold else "black")
  fig.colorbar(cax)
  #plt.colorbar()
  plt.show()

In [ ]:
plot_conf_matrix(test_generator.classes, predictedClasses, normalize = True)

In [ ]:
plot_conf_matrix(test_generator.classes, predictedClasses, normalize = False)

In [ ]:
labels = list(test_generator.class_indices.keys())

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_generator.classes, predictedClasses))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print( "Acc" + str(accuracy_score(test_generator.classes, predictedClasses)))
print( "F1" + str(f1_score(test_generator.classes, predictedClasses, average='weighted')))
print( "Precision" + str(precision_score(test_generator.classes, predictedClasses, average='weighted')))
print( " Recall" + str(recall_score(test_generator.classes, predictedClasses, average='weighted')))

In [ ]:
totalClassCounts = {}
for label in labels:
  totalClassCounts[label] = 0
for generator in [train_generator, val_generator, test_generator]:
  for key in generator.class_indices.keys():
   totalClassCounts[key] +=  generator.classes.tolist().count(generator.class_indices[key]) 

In [ ]:
totalClassCounts

In [ ]:
sum(totalClassCounts.values())

In [ ]:
plt.figure(figsize=(10,5))
plot = plt.bar(totalClassCounts.keys(), totalClassCounts.values())
for bar in plot:
    yval = bar.get_height()
    plt.text(bar.get_x() + 0.3, yval + 65, yval)

plt.show()